In [1]:
# FINAL NOTEBOOK: Perception + Pick + Throw Pipeline
# Combines pick_colored_ball_camera.ipynb and arc_trajectory_planning.ipynb
#
# Pipeline:
# 1. Setup scene with robot, table, red/blue balls, 4 cameras
# 2. Multi-camera perception to detect red ball
# 3. Pick trajectory to grasp detected ball
# 4. Throw trajectory to launch ball at target

from pydrake.all import *
from manipulation.station import LoadScenario, MakeHardwareStation
from manipulation.meshcat_utils import AddMeshcatTriad
from pydrake.geometry import Rgba, Cylinder
from pydrake.systems.sensors import RgbdSensor, CameraConfig
from pathlib import Path
import numpy as np
import tempfile
import os
import sys

sys.path.append(str(Path.cwd()))
from throw_helpers import create_q_knots, joint_angles_to_pose

print("Imports loaded")

Imports loaded


In [3]:
# Start meshcat
meshcat = StartMeshcat()
print(f"Meshcat: {meshcat.web_url()}")

INFO:drake:Meshcat listening for connections at http://localhost:7001


Meshcat: http://localhost:7001


In [4]:
# Perception helper functions

def depth_image_to_point_cloud(depth_image, rgb_image, camera_info):
    """Convert depth image to colored point cloud in camera frame"""
    height, width = depth_image.shape
    fx = camera_info.focal_x()
    fy = camera_info.focal_y()
    cx = camera_info.center_x()
    cy = camera_info.center_y()
    
    u = np.arange(width)
    v = np.arange(height)
    u_grid, v_grid = np.meshgrid(u, v)
    
    valid = (depth_image > 0.01) & (depth_image < 5.0)
    
    u_valid = u_grid[valid]
    v_valid = v_grid[valid]
    z_valid = depth_image[valid]
    
    x = (u_valid - cx) * z_valid / fx
    y = (v_valid - cy) * z_valid / fy
    z = z_valid
    
    points = np.column_stack([x, y, z])
    colors = rgb_image[valid].astype(float) / 255.0
    
    return points, colors


def transform_points_to_world(points, X_CameraToWorld):
    """Transform points from camera frame to world frame"""
    points_world = []
    for pt in points:
        pt_world = X_CameraToWorld @ pt
        points_world.append(pt_world)
    return np.array(points_world)


def make_camera_transform(camera_pos, look_at_pos):
    """Create camera transform that looks from camera_pos to look_at_pos"""
    forward = look_at_pos - camera_pos
    forward = forward / np.linalg.norm(forward)
    world_up = np.array([0, 0, 1])
    right = np.cross(forward, world_up)
    right = right / np.linalg.norm(right)
    down = np.cross(forward, right)
    R_WorldToCamera = RotationMatrix(np.column_stack([right, down, forward]))
    return RigidTransform(R_WorldToCamera, camera_pos)


print("Perception functions defined")

Perception functions defined


In [5]:
# Scene setup: Robot + Table + 2 Balls + 4 Cameras

# Create temporary SDF files for balls
temp_dir = tempfile.mkdtemp()

red_ball_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="red_ball">
    <link name="ball">
      <inertial>
        <mass>0.1</mass>
        <inertia>
          <ixx>0.0001</ixx><ixy>0</ixy><ixz>0</ixz>
          <iyy>0.0001</iyy><iyz>0</iyz><izz>0.0001</izz>
        </inertia>
      </inertial>
      <visual name="visual">
        <geometry><sphere><radius>0.05</radius></sphere></geometry>
        <material><diffuse>0.9 0.1 0.1 1.0</diffuse></material>
      </visual>
      <collision name="collision">
        <geometry><sphere><radius>0.05</radius></sphere></geometry>
      </collision>
    </link>
  </model>
</sdf>
"""

blue_ball_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="blue_ball">
    <link name="ball">
      <inertial>
        <mass>0.1</mass>
        <inertia>
          <ixx>0.0001</ixx><ixy>0</ixy><ixz>0</ixz>
          <iyy>0.0001</iyy><iyz>0</iyz><izz>0.0001</izz>
        </inertia>
      </inertial>
      <visual name="visual">
        <geometry><sphere><radius>0.05</radius></sphere></geometry>
        <material><diffuse>0.2 0.4 0.9 1.0</diffuse></material>
      </visual>
      <collision name="collision">
        <geometry><sphere><radius>0.05</radius></sphere></geometry>
      </collision>
    </link>
  </model>
</sdf>
"""

red_ball_path = os.path.join(temp_dir, "red_ball.sdf")
blue_ball_path = os.path.join(temp_dir, "blue_ball.sdf")

with open(red_ball_path, 'w') as f:
    f.write(red_ball_sdf)
with open(blue_ball_path, 'w') as f:
    f.write(blue_ball_sdf)

# Ball positions on table
ball_radius = 0.05
red_pos = np.array([0.0, -0.5, ball_radius])
blue_pos = np.array([-0.15, -0.55, ball_radius])

# Table path
abs_table_sdf_path = f"{Path.cwd()}/assets/table.sdf"

# Scenario YAML with free-body balls
scenario_yaml = f"""
directives:
- add_model:
    name: iiwa
    file: package://drake_models/iiwa_description/sdf/iiwa7_no_collision.sdf
    default_joint_positions:
      iiwa_joint_1: [-1.57]
      iiwa_joint_2: [0.1]
      iiwa_joint_3: [0]
      iiwa_joint_4: [-1.2]
      iiwa_joint_5: [0]
      iiwa_joint_6: [1.6]
      iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
- add_model:
    name: wsg
    file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
      translation: [0, 0, 0.09]
      rotation: !Rpy {{ deg: [90, 0, 90] }}
- add_model:
    name: table
    file: file://{abs_table_sdf_path}
- add_weld:
    parent: world
    child: table::table_link
    X_PC:
      translation: [0.0, 0.0, -0.05]
      rotation: !Rpy {{ deg: [0, 0, -90] }}
- add_model:
    name: red_ball
    file: file://{red_ball_path}
    default_free_body_pose:
      ball:
        translation: [{red_pos[0]}, {red_pos[1]}, {red_pos[2]}]
- add_model:
    name: blue_ball
    file: file://{blue_ball_path}
    default_free_body_pose:
      ball:
        translation: [{blue_pos[0]}, {blue_pos[1]}, {blue_pos[2]}]

model_drivers:
    iiwa: !IiwaDriver
      control_mode: position_only
      hand_model_name: wsg
    wsg: !SchunkWsgDriver {{}}
"""

# Build diagram with cameras
meshcat.Delete()
scenario = LoadScenario(data=scenario_yaml)
builder = DiagramBuilder()

station = MakeHardwareStation(scenario, meshcat=meshcat)
builder.AddSystem(station)

plant = station.GetSubsystemByName("plant")
scene_graph = station.GetSubsystemByName("scene_graph")

# Get initial gripper pose BEFORE building diagram
temp_context = station.CreateDefaultContext()
temp_plant_context = plant.GetMyContextFromRoot(temp_context)
X_WG_initial = plant.EvalBodyPoseInWorld(temp_plant_context, plant.GetBodyByName("body"))
print(f"Initial gripper pose: {X_WG_initial.translation()}")

# Add VTK renderer
renderer_name = "my_renderer"
scene_graph.AddRenderer(renderer_name, MakeRenderEngineVtk(RenderEngineVtkParams()))

# Camera config
width, height = 640, 480
camera_config = CameraConfig()
camera_config.width = width
camera_config.height = height
camera_config.fps = 10.0
camera_config.renderer_name = renderer_name

# 4 camera positions spread around workspace
workspace_center = np.array([0.0, -0.5, 0.05])
camera_positions = [
    np.array([0.4, -0.15, 0.5]),    # Front-right, HIGH
    np.array([-0.35, -0.9, 0.25]),  # Back-left, LOW
    np.array([-0.5, -0.4, 0.4]),    # Left side, MEDIUM
    np.array([0.5, -0.6, 0.35]),    # Right side
]

camera_transforms = [make_camera_transform(pos, workspace_center) for pos in camera_positions]
camera_sensors = []
camera_infos = []

for i, X_cam in enumerate(camera_transforms):
    color_cam, depth_cam = camera_config.MakeCameras()
    sensor = builder.AddSystem(RgbdSensor(
        parent_id=scene_graph.world_frame_id(),
        X_PB=X_cam,
        color_camera=color_cam,
        depth_camera=depth_cam
    ))
    builder.Connect(station.GetOutputPort("query_object"), sensor.query_object_input_port())
    camera_sensors.append(sensor)
    camera_infos.append(color_cam.core().intrinsics())

# Build diagram
diagram = builder.Build()
context = diagram.CreateDefaultContext()
diagram.ForcedPublish(context)

# Visualize cameras
cam_colors = [
    Rgba(0.0, 0.5, 1.0, 0.8),  # Cyan
    Rgba(1.0, 0.5, 0.0, 0.8),  # Orange
    Rgba(0.8, 0.0, 0.8, 0.8),  # Purple
    Rgba(0.0, 1.0, 0.5, 0.8),  # Teal
]

for i, (X_cam, color) in enumerate(zip(camera_transforms, cam_colors), 1):
    meshcat.SetObject(f"camera{i}/body", Box(0.05, 0.05, 0.08), color)
    meshcat.SetTransform(f"camera{i}/body", X_cam)

print(f"\nScene setup complete:")
print(f"  Red ball: {red_pos}")
print(f"  Blue ball: {blue_pos}")
print(f"  4 cameras positioned around workspace")

INFO:drake:PackageMap: Downloading https://github.com/RobotLocomotion/models/archive/7b92aacbe021861ec9bbbb82d8ab9a19ded970ff.tar.gz


Initial gripper pose: [ 3.70783219e-04 -4.65616808e-01  6.79321579e-01]

Scene setup complete:
  Red ball: [ 0.   -0.5   0.05]
  Blue ball: [-0.15 -0.55  0.05]
  4 cameras positioned around workspace


In [11]:
# PERCEPTION: Multi-camera capture and point cloud fusion (4 CAMERAS)

print("=== MULTI-CAMERA PERCEPTION PIPELINE ===\n")

# Lists to store point clouds from all cameras
all_points_world = []
all_colors = []

# Process each camera
for cam_idx, (sensor, cam_info, X_cam) in enumerate(zip(camera_sensors, camera_infos, camera_transforms), 1):
    print(f"--- Camera {cam_idx} ---")
    
    # Get images from this sensor
    sensor_context = sensor.GetMyContextFromRoot(context)
    
    color_image = sensor.color_image_output_port().Eval(sensor_context)
    rgb_img = color_image.data[:, :, :3].astype(np.uint8)
    
    depth_image = sensor.depth_image_32F_output_port().Eval(sensor_context)
    depth_img = depth_image.data.squeeze().astype(np.float32)
    
    # Convert to point cloud in camera frame
    points_camera, colors = depth_image_to_point_cloud(depth_img, rgb_img, cam_info)
    print(f"  Generated {len(points_camera)} points")
    
    # Transform to world frame
    points_world = transform_points_to_world(points_camera, X_cam)
    
    all_points_world.append(points_world)
    all_colors.append(colors)

# Merge all point clouds
merged_points = np.vstack(all_points_world)
merged_colors = np.vstack(all_colors)

print(f"\n✓ Merged {len(merged_points)} points from {len(camera_sensors)} cameras\n")

# VISUALIZE point clouds from each camera with DIFFERENT COLORS
print("--- Visualizing Point Clouds (color-coded by camera) ---")
cam_viz_colors = [
    Rgba(0.0, 0.5, 1.0, 0.9),   # Cyan - Camera 1
    Rgba(1.0, 0.5, 0.0, 0.9),   # Orange - Camera 2  
    Rgba(0.8, 0.0, 0.8, 0.9),   # Purple - Camera 3
    Rgba(0.0, 1.0, 0.5, 0.9),   # Teal - Camera 4
]
cam_names = ['Cyan', 'Orange', 'Purple', 'Teal']

# Downsample for visualization
downsample = 300

for cam_idx, (points, viz_color) in enumerate(zip(all_points_world, cam_viz_colors)):
    print(f"  Camera {cam_idx+1}: plotting {len(points)//downsample} points ({cam_names[cam_idx]})")
    for i in range(0, len(points), downsample):
        meshcat.SetObject(f"pointcloud/cam{cam_idx+1}/pt_{i}", Sphere(0.004), viz_color)
        meshcat.SetTransform(f"pointcloud/cam{cam_idx+1}/pt_{i}", RigidTransform(points[i]))

print(f"  ✓ Plotted ~{len(merged_points)//downsample} points total")
print(f"  Legend: Cyan=Cam1, Orange=Cam2, Purple=Cam3, Teal=Cam4\n")

# BALL DETECTION
print("--- Ball Detection ---")

# Step 1: Find points that are ABOVE the table (z > 0.02)
above_table_mask = merged_points[:, 2] > 0.02
above_table_points = merged_points[above_table_mask]
above_table_colors = merged_colors[above_table_mask]
print(f"Points above table (z > 0.02): {len(above_table_points)}")

# Step 2: Among elevated points, find the reddest ones
if len(above_table_points) > 0:
    redness = above_table_colors[:, 0] - np.maximum(above_table_colors[:, 1], above_table_colors[:, 2])
    
    # Explicitly cast to Python int to avoid numpy int64 indexing issues
    num_red_points = int(max(100, int(len(above_table_points) * 0.05)))
    red_threshold = np.sort(redness)[-num_red_points]
    
    is_red = redness >= red_threshold
    red_points = above_table_points[is_red]
    print(f"Reddest {num_red_points} elevated points selected")
    
    # Step 3: Cluster to find ball center
    if len(red_points) > 10:
        centroid = np.mean(red_points, axis=0)
        
        for iteration in range(3):
            distances = np.linalg.norm(red_points - centroid, axis=1)
            inliers = distances < 0.06
            if np.sum(inliers) > 10:
                centroid = np.mean(red_points[inliers], axis=0)
                red_points = red_points[inliers]
        
        print(f"After clustering: {len(red_points)} points")
        
        detected_red_pos = np.mean(red_points, axis=0)
        detected_red_pos[2] = ball_radius
        
        error = np.linalg.norm(detected_red_pos - red_pos)
        print(f"\nDetected: {detected_red_pos}")
        print(f"Ground truth: {red_pos}")
        print(f"Error: {error*1000:.1f}mm")
        
        # Visualize detected ball (bright yellow)
        meshcat.SetObject("perception/detected", Sphere(0.055), Rgba(1.0, 1.0, 0.0, 0.6))
        meshcat.SetTransform("perception/detected", RigidTransform(detected_red_pos))
        
        # Visualize clustered red points (bright green for visibility)
        step = int(max(1, len(red_points) // 30))
        for i in range(0, len(red_points), step):
            meshcat.SetObject(f"perception/red_pt_{i}", Sphere(0.008), Rgba(0, 1, 0, 1.0))
            meshcat.SetTransform(f"perception/red_pt_{i}", RigidTransform(red_points[i]))
        
        print("\n" + "="*60)
        print(f"✓ DETECTED RED BALL - Error: {error*1000:.1f}mm")
        print("="*60)
    else:
        detected_red_pos = None
        print("✗ Too few red points for clustering")
else:
    detected_red_pos = None
    print("✗ No elevated points found")

=== MULTI-CAMERA PERCEPTION PIPELINE ===

--- Camera 1 ---
  Generated 266226 points
--- Camera 2 ---
  Generated 246300 points
--- Camera 3 ---
  Generated 270377 points
--- Camera 4 ---
  Generated 270942 points

✓ Merged 1053845 points from 4 cameras

--- Visualizing Point Clouds (color-coded by camera) ---
  Camera 1: plotting 887 points (Cyan)
  Camera 2: plotting 821 points (Orange)
  Camera 3: plotting 901 points (Purple)
  Camera 4: plotting 903 points (Teal)
  ✓ Plotted ~3512 points total
  Legend: Cyan=Cam1, Orange=Cam2, Purple=Cam3, Teal=Cam4

--- Ball Detection ---
Points above table (z > 0.02): 83671
Reddest 4183 elevated points selected
After clustering: 4244 points

Detected: [-0.00219639 -0.50580673  0.05      ]
Ground truth: [ 0.   -0.5   0.05]
Error: 6.2mm

✓ DETECTED RED BALL - Error: 6.2mm


In [12]:
# TRAJECTORY PLANNING: Pick + Throw

if detected_red_pos is None:
    raise RuntimeError("Cannot plan trajectory - no ball detected")

print("=== TRAJECTORY PLANNING ===")

# Grasp parameters
GRIPPER_OPEN = 0.107
GRIPPER_CLOSED = 0.0
GRASP_HEIGHT = 0.12
APPROACH_HEIGHT = 0.1

# Target for throwing
P_TARGET = np.array([0, 1.0, 0.0])
RELEASE_FRAC = 0.5
THROW_DURATION = 0.4

def design_grasp_pose(X_WO: RigidTransform) -> RigidTransform:
    """Grasp pose above object with gripper pointing down."""
    return X_WO @ RigidTransform(RotationMatrix.MakeXRotation(-np.pi/2), [0, 0, GRASP_HEIGHT])

# Ball pose from perception
X_WBall = RigidTransform(p=detected_red_pos)

# Pick trajectory poses
X_WG_grasp = design_grasp_pose(X_WBall)
X_WG_approach = RigidTransform(p=[0, 0, APPROACH_HEIGHT]) @ X_WG_grasp

print(f"Target ball: {detected_red_pos}")
print(f"Grasp pose: {X_WG_grasp.translation()}")
print(f"Throw target: {P_TARGET}")

# Pick timing
t_approach = 1.0
t_descend = 0.5
t_grasp = 0.5
t_lift = 0.5

t0, t1, t2, t3, t4 = 0, t_approach, t_approach + t_descend, t_approach + t_descend + t_grasp, t_approach + t_descend + t_grasp + t_lift

# Create pick pose trajectory
pose_traj = PiecewisePose.MakeLinear(
    [t0, t1, t2, t3, t4],
    [X_WG_initial, X_WG_approach, X_WG_grasp, X_WG_grasp, X_WG_approach]
)

# Gripper trajectory for pick
pick_gripper_traj = PiecewisePolynomial.FirstOrderHold(
    [t0, t1, t2, t3, t4],
    np.array([[GRIPPER_OPEN, GRIPPER_OPEN, GRIPPER_OPEN, GRIPPER_CLOSED, GRIPPER_CLOSED]])
)

# Convert pick poses to joint angles
print("\nSolving IK for pick trajectory...")
num_samples = 50
t_samples = np.linspace(t0, t4, num_samples)
poses = [pose_traj.GetPose(t) for t in t_samples]
q_pick_knots = create_q_knots(plant, poses)
q_pick_traj = PiecewisePolynomial.CubicShapePreserving(t_samples[:len(q_pick_knots)], np.array(q_pick_knots).T)
print(f"Pick trajectory: {q_pick_traj.end_time():.2f}s")

# Throw trajectory
throw_heading = np.arctan2(P_TARGET[1], P_TARGET[0])
ja1 = throw_heading - np.pi

PRETHROW_JA = np.array([ja1, 0, 0, 1.9, 0, -1.9, np.pi])
THROWEND_JA = np.array([ja1, 0, 0, 0.4, 0, -0.4, np.pi])

q_postpick = q_pick_traj.value(q_pick_traj.end_time()).flatten()

def interpolate_joint_angles(ja_start, ja_end, duration, num_samples):
    t_samples = np.linspace(0, duration, num_samples, endpoint=True)
    q_samples = [ja_start + (t / duration) * (ja_end - ja_start) for t in t_samples]
    return t_samples, q_samples

t_throw_samples, q_throw_samples = interpolate_joint_angles(PRETHROW_JA, THROWEND_JA, THROW_DURATION, 30)

# Combine pick + throw trajectories
T_GO_TO_PRETHROW = 1.5
t_pick_end = q_pick_traj.end_time()

t_full = list(np.linspace(0, t_pick_end, 50))
t_full += [t_pick_end + T_GO_TO_PRETHROW]
t_full += list(t_pick_end + T_GO_TO_PRETHROW + t_throw_samples[1:])

q_full = [q_pick_traj.value(t).flatten() for t in np.linspace(0, t_pick_end, 50)]
q_full += [PRETHROW_JA] + q_throw_samples[1:]

q_full_traj = PiecewisePolynomial.CubicShapePreserving(t_full, np.array(q_full).T)

# Full gripper trajectory: open -> close at grasp -> open at release
t_grasp_close = t3  # When gripper closes during pick
t_release = t_pick_end + T_GO_TO_PRETHROW + RELEASE_FRAC * THROW_DURATION

g_full_traj = PiecewisePolynomial.FirstOrderHold(
    [0, t_grasp_close, t_grasp_close + 0.01, t_release, t_release + 0.01, q_full_traj.end_time()],
    np.array([[GRIPPER_OPEN, GRIPPER_OPEN, GRIPPER_CLOSED, GRIPPER_CLOSED, GRIPPER_OPEN, GRIPPER_OPEN]])
)

print(f"\nFull trajectory: {q_full_traj.end_time():.2f}s")
print(f"  Pick: 0 - {t_pick_end:.2f}s")
print(f"  Move to prethrow: {t_pick_end:.2f} - {t_pick_end + T_GO_TO_PRETHROW:.2f}s")
print(f"  Throw: {t_pick_end + T_GO_TO_PRETHROW:.2f} - {q_full_traj.end_time():.2f}s")
print(f"  Release at: {t_release:.2f}s")

=== TRAJECTORY PLANNING ===
Target ball: [-0.00219639 -0.50580673  0.05      ]
Grasp pose: [-0.00219639 -0.50580673  0.17      ]
Throw target: [0. 1. 0.]

Solving IK for pick trajectory...
Pick trajectory: 2.50s

Full trajectory: 4.40s
  Pick: 0 - 2.50s
  Move to prethrow: 2.50 - 4.00s
  Throw: 4.00 - 4.40s
  Release at: 4.20s


In [13]:
# EXECUTE: Full pick + throw with physics simulation

print("=== PHYSICS SIMULATION ===")

meshcat.Delete()

# Rebuild diagram with trajectory sources
builder2 = DiagramBuilder()
scenario2 = LoadScenario(data=scenario_yaml)
station2 = MakeHardwareStation(scenario2, meshcat=meshcat)
builder2.AddSystem(station2)
full_plant = station2.GetSubsystemByName("plant")

# Add triads for visualization
AddMeshcatTriad(meshcat, "target", length=0.2, radius=0.01, X_PT=RigidTransform(p=P_TARGET))
AddMeshcatTriad(meshcat, "prethrow", length=0.12, radius=0.005, X_PT=joint_angles_to_pose(full_plant, PRETHROW_JA))
AddMeshcatTriad(meshcat, "throwend", length=0.12, radius=0.005, X_PT=joint_angles_to_pose(full_plant, THROWEND_JA))

# Connect trajectory sources
q_source = builder2.AddSystem(TrajectorySource(q_full_traj))
g_source = builder2.AddSystem(TrajectorySource(g_full_traj))

builder2.Connect(q_source.get_output_port(), station2.GetInputPort("iiwa.position"))
builder2.Connect(g_source.get_output_port(), station2.GetInputPort("wsg.position"))

diagram2 = builder2.Build()

# Run simulation
simulator = Simulator(diagram2)
simulator.set_target_realtime_rate(1.0)

print(f"Running simulation...")
print(f"  Gripper closes at t={t_grasp_close:.2f}s")
print(f"  Release at t={t_release:.2f}s")

meshcat.StartRecording()
simulator.AdvanceTo(q_full_traj.end_time() + 2.0)
meshcat.StopRecording()
meshcat.PublishRecording()

print("\n" + "="*60)
print("SIMULATION COMPLETE!")
print(f"  Detected ball at: {detected_red_pos}")
print(f"  Threw towards: {P_TARGET}")
print("="*60)

=== PHYSICS SIMULATION ===
Running simulation...
  Gripper closes at t=2.00s
  Release at t=4.20s

SIMULATION COMPLETE!
  Detected ball at: [-0.00219639 -0.50580673  0.05      ]
  Threw towards: [0. 1. 0.]
